In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline
from sklearn import set_config; set_config(display = "diagram")
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.pipeline import FeatureUnion

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('final_le_wagon_project_data.csv')

In [3]:
df.head(5)

,sr_no,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months,shortlist_uni_1,...,shortlist_uni_2,admit_uni_2,shortlist_uni_3,admit_uni_3,shortlist_uni_4,admit_uni_4,shortlist_uni_5,admit_uni_5,shortlist_uni_6,admit_uni_6
0,1,Male,General,82.0,86.5,77.4,Engineer,92.43,30,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Female,General,95.0,95.6,85.4,Engineer,94.29,16,0,...,0,0,0,0,0,0,1,0,0,0
2,3,Female,General,93.1,91.6,79.2,Non-Engineer,92.52,4,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Male,General,91.8,89.7,80.7,Non-Engineer,96.57,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Female,General,96.0,86.2,92.4,Non-Engineer,91.00,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.drop(['sr_no', 'admit_uni_1', 'admit_uni_2', 'admit_uni_3', 'admit_uni_4', 'admit_uni_5', 'admit_uni_6'], axis=1)
df = df.dropna().copy()

In [5]:
df.head(10)

,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months,shortlist_uni_1,shortlist_uni_2,shortlist_uni_3,shortlist_uni_4,shortlist_uni_5,shortlist_uni_6
0,Male,General,82.0,86.5,77.4,Engineer,92.43,30,0,0,0,0,0,0
1,Female,General,95.0,95.6,85.4,Engineer,94.29,16,0,0,0,0,1,0
2,Female,General,93.1,91.6,79.2,Non-Engineer,92.52,4,0,0,0,0,0,0
3,Male,General,91.8,89.7,80.7,Non-Engineer,96.57,0,0,0,0,0,0,0
4,Female,General,96.0,86.2,92.4,Non-Engineer,91.00,0,0,0,0,0,0,0
5,Male,General,87.4,92.6,79.9,Engineer,99.63,21,0,0,1,0,0,0
6,Male,NC_OBC,99.3,89.7,73.6,Engineer,95.01,25,0,1,0,1,1,1
7,Male,General,95.0,86.8,71.1,Engineer,99.26,26,0,0,0,0,0,0
8,Male,General,87.4,79.6,70.1,Engineer,99.22,28,0,0,0,0,0,1
9,Female,NC_OBC,85.5,84.3,70.9,Non-Engineer,91.50,14,0,0,0,0,1,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3940 entries, 0 to 3939
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gender                3940 non-null   object 
 1   category              3940 non-null   object 
 2   grade_10              3940 non-null   float64
 3   grade_12              3940 non-null   float64
 4   graduation_score      3940 non-null   float64
 5   graduation_type       3940 non-null   object 
 6   admission_test_score  3940 non-null   float64
 7   work_exp_months       3940 non-null   int64  
 8   shortlist_uni_1       3940 non-null   int64  
 9   shortlist_uni_2       3940 non-null   int64  
 10  shortlist_uni_3       3940 non-null   int64  
 11  shortlist_uni_4       3940 non-null   int64  
 12  shortlist_uni_5       3940 non-null   int64  
 13  shortlist_uni_6       3940 non-null   int64  
dtypes: float64(4), int64(7), object(3)
memory usage: 431.1+ KB


In [7]:
final_preprocessor = ColumnTransformer([
    ('num_encoder', RobustScaler(), make_column_selector(dtype_exclude="object")),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore', drop = 'first', sparse=False), make_column_selector(dtype_include="object"))])

In [8]:
final_preprocessor

ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                ('cat_encoder',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])

In [9]:
X = df.drop(columns=["shortlist_uni_1", "shortlist_uni_2", "shortlist_uni_3",
                    "shortlist_uni_4", "shortlist_uni_5", "shortlist_uni_6" ])

y1 = df["shortlist_uni_1"]
y2 = df["shortlist_uni_2"]
y3 = df["shortlist_uni_3"]
y4 = df["shortlist_uni_4"]
y5 = df["shortlist_uni_5"]
y6 = df["shortlist_uni_6"]

In [10]:
df.head()

,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months,shortlist_uni_1,shortlist_uni_2,shortlist_uni_3,shortlist_uni_4,shortlist_uni_5,shortlist_uni_6
0,Male,General,82.0,86.5,77.4,Engineer,92.43,30,0,0,0,0,0,0
1,Female,General,95.0,95.6,85.4,Engineer,94.29,16,0,0,0,0,1,0
2,Female,General,93.1,91.6,79.2,Non-Engineer,92.52,4,0,0,0,0,0,0
3,Male,General,91.8,89.7,80.7,Non-Engineer,96.57,0,0,0,0,0,0,0
4,Female,General,96.0,86.2,92.4,Non-Engineer,91.00,0,0,0,0,0,0,0


In [11]:
pd.DataFrame(final_preprocessor.fit_transform(X)).head()

,0,1,2,3,4,5,6,7,8,9,10
0,-1.368421,-0.250000,-0.008197,-0.321195,1.666667,1.0,1.0,0.0,0.0,0.0,0.0
1,0.342105,0.592593,0.647541,-0.089636,0.888889,0.0,1.0,0.0,0.0,0.0,0.0
2,0.092105,0.222222,0.139344,-0.309991,0.222222,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.078947,0.046296,0.262295,0.194211,0.000000,1.0,1.0,0.0,0.0,0.0,1.0
4,0.473684,-0.277778,1.221311,-0.499222,0.000000,0.0,1.0,0.0,0.0,0.0,1.0


In [12]:
# Creating training and testing datasets
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)
X_train, X_test, y3_train, y3_test = train_test_split(X, y3, test_size=0.2, random_state=42)
X_train, X_test, y4_train, y4_test = train_test_split(X, y4, test_size=0.2, random_state=42)
X_train, X_test, y5_train, y5_test = train_test_split(X, y5, test_size=0.2, random_state=42)
X_train, X_test, y6_train, y6_test = train_test_split(X, y6, test_size=0.2, random_state=42)

Pipe 1

In [13]:
# Chain the final_preprocessor pipeline with a logistic regression model
pipe1 = Pipeline([
    ('preprocessing', final_preprocessor),
    ('logistic_regression', LogisticRegression(class_weight='balanced'))
])

pipe1

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [14]:
pipe1.fit(X_train, y1_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [15]:
pipe1['preprocessing'].transformers_

[('num_encoder',
  RobustScaler(),
  ['grade_10',
   'grade_12',
   'graduation_score',
   'admission_test_score',
   'work_exp_months']),
 ('cat_encoder',
  OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False,
                sparse_output=False),
  ['gender', 'category', 'graduation_type'])]

In [16]:
col_num = pipe1['preprocessing'].transformers_[0][2]
col_num

['grade_10',
 'grade_12',
 'graduation_score',
 'admission_test_score',
 'work_exp_months']

In [17]:
pipe1['preprocessing'].transformers_[1][1].get_feature_names_out()

array(['gender_Male', 'category_General', 'category_NC_OBC',
       'category_SC', 'category_ST', 'graduation_type_Non-Engineer'],
      dtype=object)

In [18]:
col_cat = list(pipe1['preprocessing'].transformers_[1][1].get_feature_names_out())
col_cat

['gender_Male',
 'category_General',
 'category_NC_OBC',
 'category_SC',
 'category_ST',
 'graduation_type_Non-Engineer']

In [19]:
import itertools
col_preproc = list(itertools.chain(col_num, col_cat))

In [20]:
len(col_preproc)

11

In [21]:
pd.DataFrame(data=pipe1['preprocessing'].fit_transform(X_train), columns=col_preproc).head()

,grade_10,grade_12,graduation_score,admission_test_score,work_exp_months,gender_Male,category_General,category_NC_OBC,category_SC,category_ST,graduation_type_Non-Engineer
0,-0.098684,-0.568765,-0.720648,-0.618425,0.944444,1.0,0.0,0.0,1.0,0.0,0.0
1,-0.914474,0.428904,-0.874494,-3.766656,1.444444,0.0,1.0,0.0,0.0,0.0,1.0
2,-1.717105,-0.466200,-1.255061,-1.579853,0.000000,0.0,0.0,1.0,0.0,0.0,1.0
3,0.361842,0.447552,1.157895,-0.867070,0.333333,1.0,1.0,0.0,0.0,0.0,0.0
4,0.677632,0.634033,0.518219,0.517692,1.222222,1.0,1.0,0.0,0.0,0.0,0.0


In [22]:
scores1 = cross_val_score(pipe1, X_train, y1_train, cv=5, scoring="accuracy")
score1 = scores1.mean()

print(scores1)
print(scores1.mean())

[0.69096672 0.69730586 0.68571429 0.7015873  0.68253968]
0.6916227706085076


In [23]:
import pickle

# Export pipeline as pickle file
with open("pipeline1.pkl", "wb") as file:
    pickle.dump(pipe1, file)

In [24]:
# Load pipeline from pickle file
my_pipeline1 = pickle.load(open("pipeline1.pkl", "rb"))

my_pipeline1

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dddb310>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dddb010>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [25]:
new_data = pd.read_csv("/Users/prasadsawant/Downloads/new_data.csv")

In [26]:
new_data

,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months
0,Male,General,68.16,57.58,59.10,Engineer,77.04,26
1,Male,ST,89.75,79.13,92.36,Non-Engineer,94.16,23
2,Female,SC,74.81,81.42,87.84,Non-Engineer,84.27,17
3,Female,ST,91.75,68.73,58.49,Engineer,71.17,2
4,Female,EWS,62.55,79.76,58.94,Non-Engineer,88.75,26
5,Female,SC,69.22,59.70,88.16,Engineer,93.41,19
6,Male,SC,68.38,80.86,78.22,Non-Engineer,94.47,9
7,Male,General,56.94,76.89,69.00,Non-Engineer,61.67,10
8,Male,General,85.90,87.64,95.31,Engineer,95.79,0
9,Female,General,87.70,75.37,78.85,Non-Engineer,60.86,27


In [27]:
predicted_class_1 = my_pipeline1.predict(new_data)
predicted_class_1

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0])

Pipe 2

In [28]:
# Chain the final_preprocessor pipeline with a logistic regression model
pipe2 = Pipeline([
    ('preprocessing', final_preprocessor),
    ('logistic_regression', LogisticRegression(class_weight='balanced'))
])

pipe2

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [29]:
pipe2.fit(X_train, y2_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [30]:
pipe2['preprocessing'].transformers_

[('num_encoder',
  RobustScaler(),
  ['grade_10',
   'grade_12',
   'graduation_score',
   'admission_test_score',
   'work_exp_months']),
 ('cat_encoder',
  OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False,
                sparse_output=False),
  ['gender', 'category', 'graduation_type'])]

In [31]:
col_num = pipe2['preprocessing'].transformers_[0][2]
col_num

['grade_10',
 'grade_12',
 'graduation_score',
 'admission_test_score',
 'work_exp_months']

In [32]:
pipe2['preprocessing'].transformers_[1][1].get_feature_names_out()

array(['gender_Male', 'category_General', 'category_NC_OBC',
       'category_SC', 'category_ST', 'graduation_type_Non-Engineer'],
      dtype=object)

In [33]:
col_cat = list(pipe2['preprocessing'].transformers_[1][1].get_feature_names_out())
col_cat

['gender_Male',
 'category_General',
 'category_NC_OBC',
 'category_SC',
 'category_ST',
 'graduation_type_Non-Engineer']

In [34]:
import itertools
col_preproc = list(itertools.chain(col_num, col_cat))

In [35]:
len(col_preproc)

11

In [36]:
pd.DataFrame(data=pipe2['preprocessing'].fit_transform(X_train), columns=col_preproc).head()

,grade_10,grade_12,graduation_score,admission_test_score,work_exp_months,gender_Male,category_General,category_NC_OBC,category_SC,category_ST,graduation_type_Non-Engineer
0,-0.098684,-0.568765,-0.720648,-0.618425,0.944444,1.0,0.0,0.0,1.0,0.0,0.0
1,-0.914474,0.428904,-0.874494,-3.766656,1.444444,0.0,1.0,0.0,0.0,0.0,1.0
2,-1.717105,-0.466200,-1.255061,-1.579853,0.000000,0.0,0.0,1.0,0.0,0.0,1.0
3,0.361842,0.447552,1.157895,-0.867070,0.333333,1.0,1.0,0.0,0.0,0.0,0.0
4,0.677632,0.634033,0.518219,0.517692,1.222222,1.0,1.0,0.0,0.0,0.0,0.0


In [37]:
scores2 = cross_val_score(pipe2, X_train, y2_train, cv=5, scoring="accuracy")
score2 = scores2.mean()

print(scores2)
print(scores2.mean())

[0.75911252 0.7163233  0.74920635 0.76349206 0.77142857]
0.7519125600583603


In [38]:
import pickle

# Export pipeline as pickle file
with open("pipeline2.pkl", "wb") as file:
    pickle.dump(pipe2, file)

In [39]:
# Load pipeline from pickle file
my_pipeline2 = pickle.load(open("pipeline2.pkl", "rb"))

my_pipeline2

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11de095d0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11de08730>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [40]:
predicted_class_2 = my_pipeline2.predict(new_data)
predicted_class_2

array([0, 1, 1, 0, 0, 1, 1, 0, 0, 0])

Pipe 3

In [41]:
# Chain the final_preprocessor pipeline with a logistic regression model
pipe3 = Pipeline([
    ('preprocessing', final_preprocessor),
    ('logistic_regression', LogisticRegression(class_weight='balanced'))
])

pipe3

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [42]:
pipe3.fit(X_train, y3_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [43]:
pipe3['preprocessing'].transformers_

[('num_encoder',
  RobustScaler(),
  ['grade_10',
   'grade_12',
   'graduation_score',
   'admission_test_score',
   'work_exp_months']),
 ('cat_encoder',
  OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False,
                sparse_output=False),
  ['gender', 'category', 'graduation_type'])]

In [44]:
col_num = pipe3['preprocessing'].transformers_[0][2]
col_num

['grade_10',
 'grade_12',
 'graduation_score',
 'admission_test_score',
 'work_exp_months']

In [45]:
pipe3['preprocessing'].transformers_[1][1].get_feature_names_out()

array(['gender_Male', 'category_General', 'category_NC_OBC',
       'category_SC', 'category_ST', 'graduation_type_Non-Engineer'],
      dtype=object)

In [46]:
col_cat = list(pipe3['preprocessing'].transformers_[1][1].get_feature_names_out())
col_cat

['gender_Male',
 'category_General',
 'category_NC_OBC',
 'category_SC',
 'category_ST',
 'graduation_type_Non-Engineer']

In [47]:
import itertools
col_preproc = list(itertools.chain(col_num, col_cat))

In [48]:
len(col_preproc)

11

In [49]:
pd.DataFrame(data=pipe3['preprocessing'].fit_transform(X_train), columns=col_preproc).head()

,grade_10,grade_12,graduation_score,admission_test_score,work_exp_months,gender_Male,category_General,category_NC_OBC,category_SC,category_ST,graduation_type_Non-Engineer
0,-0.098684,-0.568765,-0.720648,-0.618425,0.944444,1.0,0.0,0.0,1.0,0.0,0.0
1,-0.914474,0.428904,-0.874494,-3.766656,1.444444,0.0,1.0,0.0,0.0,0.0,1.0
2,-1.717105,-0.466200,-1.255061,-1.579853,0.000000,0.0,0.0,1.0,0.0,0.0,1.0
3,0.361842,0.447552,1.157895,-0.867070,0.333333,1.0,1.0,0.0,0.0,0.0,0.0
4,0.677632,0.634033,0.518219,0.517692,1.222222,1.0,1.0,0.0,0.0,0.0,0.0


In [50]:
scores3 = cross_val_score(pipe3, X_train, y3_train, cv=5, scoring="accuracy")
score3 = scores3.mean()

print(scores3)
print(scores3.mean())

[0.78605388 0.77020602 0.81428571 0.77142857 0.78253968]
0.784902774633361


In [51]:
import pickle

# Export pipeline as pickle file
with open("pipeline3.pkl", "wb") as file:
    pickle.dump(pipe3, file)

In [52]:
# Load pipeline from pickle file
my_pipeline3 = pickle.load(open("pipeline3.pkl", "rb"))

my_pipeline3

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11de09630>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11de0ac20>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [53]:
predicted_class_3 = my_pipeline3.predict(new_data)
predicted_class_3

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 0])

Pipe 4

In [54]:
# Chain the final_preprocessor pipeline with a logistic regression model
pipe4 = Pipeline([
    ('preprocessing', final_preprocessor),
    ('logistic_regression', LogisticRegression(class_weight='balanced'))
])

pipe4

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [55]:
pipe4.fit(X_train, y4_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [56]:
pipe4['preprocessing'].transformers_

[('num_encoder',
  RobustScaler(),
  ['grade_10',
   'grade_12',
   'graduation_score',
   'admission_test_score',
   'work_exp_months']),
 ('cat_encoder',
  OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False,
                sparse_output=False),
  ['gender', 'category', 'graduation_type'])]

In [57]:
col_num = pipe4['preprocessing'].transformers_[0][2]
col_num

['grade_10',
 'grade_12',
 'graduation_score',
 'admission_test_score',
 'work_exp_months']

In [58]:
pipe4['preprocessing'].transformers_[1][1].get_feature_names_out()

array(['gender_Male', 'category_General', 'category_NC_OBC',
       'category_SC', 'category_ST', 'graduation_type_Non-Engineer'],
      dtype=object)

In [59]:
col_cat = list(pipe4['preprocessing'].transformers_[1][1].get_feature_names_out())
col_cat

['gender_Male',
 'category_General',
 'category_NC_OBC',
 'category_SC',
 'category_ST',
 'graduation_type_Non-Engineer']

In [60]:
import itertools
col_preproc = list(itertools.chain(col_num, col_cat))

In [61]:
len(col_preproc)

11

In [62]:
pd.DataFrame(data=pipe4['preprocessing'].fit_transform(X_train), columns=col_preproc).head()

,grade_10,grade_12,graduation_score,admission_test_score,work_exp_months,gender_Male,category_General,category_NC_OBC,category_SC,category_ST,graduation_type_Non-Engineer
0,-0.098684,-0.568765,-0.720648,-0.618425,0.944444,1.0,0.0,0.0,1.0,0.0,0.0
1,-0.914474,0.428904,-0.874494,-3.766656,1.444444,0.0,1.0,0.0,0.0,0.0,1.0
2,-1.717105,-0.466200,-1.255061,-1.579853,0.000000,0.0,0.0,1.0,0.0,0.0,1.0
3,0.361842,0.447552,1.157895,-0.867070,0.333333,1.0,1.0,0.0,0.0,0.0,0.0
4,0.677632,0.634033,0.518219,0.517692,1.222222,1.0,1.0,0.0,0.0,0.0,0.0


In [63]:
scores4 = cross_val_score(pipe4, X_train, y4_train, cv=5, scoring="accuracy")
score4 = scores4.mean()

print(scores4)
print(scores4.mean())

[0.71790808 0.72107765 0.72222222 0.73809524 0.74920635]
0.7297019092898649


In [64]:
import pickle

# Export pipeline as pickle file
with open("pipeline4.pkl", "wb") as file:
    pickle.dump(pipe4, file)

In [65]:
# Load pipeline from pickle file
my_pipeline4 = pickle.load(open("pipeline4.pkl", "rb"))

my_pipeline4

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11de44430>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11de44490>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [66]:
predicted_class_4 = my_pipeline4.predict(new_data)
predicted_class_4

array([0, 1, 1, 0, 0, 0, 1, 0, 0, 0])

Pipe 5

In [67]:
# Chain the final_preprocessor pipeline with a logistic regression model
pipe5 = Pipeline([
    ('preprocessing', final_preprocessor),
    ('logistic_regression', LogisticRegression(class_weight='balanced'))
])

pipe5

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [68]:
pipe5.fit(X_train, y5_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [69]:
pipe5['preprocessing'].transformers_

[('num_encoder',
  RobustScaler(),
  ['grade_10',
   'grade_12',
   'graduation_score',
   'admission_test_score',
   'work_exp_months']),
 ('cat_encoder',
  OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False,
                sparse_output=False),
  ['gender', 'category', 'graduation_type'])]

In [70]:
col_num = pipe5['preprocessing'].transformers_[0][2]
col_num

['grade_10',
 'grade_12',
 'graduation_score',
 'admission_test_score',
 'work_exp_months']

In [71]:
pipe5['preprocessing'].transformers_[1][1].get_feature_names_out()

array(['gender_Male', 'category_General', 'category_NC_OBC',
       'category_SC', 'category_ST', 'graduation_type_Non-Engineer'],
      dtype=object)

In [72]:
col_cat = list(pipe5['preprocessing'].transformers_[1][1].get_feature_names_out())
col_cat

['gender_Male',
 'category_General',
 'category_NC_OBC',
 'category_SC',
 'category_ST',
 'graduation_type_Non-Engineer']

In [73]:
import itertools
col_preproc = list(itertools.chain(col_num, col_cat))

In [74]:
len(col_preproc)

11

In [75]:
pd.DataFrame(data=pipe5['preprocessing'].fit_transform(X_train), columns=col_preproc).head()

,grade_10,grade_12,graduation_score,admission_test_score,work_exp_months,gender_Male,category_General,category_NC_OBC,category_SC,category_ST,graduation_type_Non-Engineer
0,-0.098684,-0.568765,-0.720648,-0.618425,0.944444,1.0,0.0,0.0,1.0,0.0,0.0
1,-0.914474,0.428904,-0.874494,-3.766656,1.444444,0.0,1.0,0.0,0.0,0.0,1.0
2,-1.717105,-0.466200,-1.255061,-1.579853,0.000000,0.0,0.0,1.0,0.0,0.0,1.0
3,0.361842,0.447552,1.157895,-0.867070,0.333333,1.0,1.0,0.0,0.0,0.0,0.0
4,0.677632,0.634033,0.518219,0.517692,1.222222,1.0,1.0,0.0,0.0,0.0,0.0


In [76]:
scores5 = cross_val_score(pipe5, X_train, y5_train, cv=5, scoring="accuracy")
score5 = scores5.mean()

print(scores5)
print(scores5.mean())

[0.79714739 0.76228209 0.75396825 0.76507937 0.75555556]
0.7668065303247553


In [77]:
import pickle

# Export pipeline as pickle file
with open("pipeline5.pkl", "wb") as file:
    pickle.dump(pipe5, file)

In [78]:
# Load pipeline from pickle file
my_pipeline5 = pickle.load(open("pipeline5.pkl", "rb"))

my_pipeline5

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11de70640>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11de70c10>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [79]:
predicted_class_5 = my_pipeline5.predict(new_data)
predicted_class_5

array([0, 1, 1, 1, 1, 1, 1, 0, 0, 0])

Pipe 6

In [80]:
# Chain the final_preprocessor pipeline with a logistic regression model
pipe6 = Pipeline([
    ('preprocessing', final_preprocessor),
    ('logistic_regression', LogisticRegression(class_weight='balanced'))
])

pipe6

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [81]:
pipe6.fit(X_train, y6_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd882e0>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd88400>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [82]:
pipe6['preprocessing'].transformers_

[('num_encoder',
  RobustScaler(),
  ['grade_10',
   'grade_12',
   'graduation_score',
   'admission_test_score',
   'work_exp_months']),
 ('cat_encoder',
  OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False,
                sparse_output=False),
  ['gender', 'category', 'graduation_type'])]

In [83]:
col_num = pipe6['preprocessing'].transformers_[0][2]
col_num

['grade_10',
 'grade_12',
 'graduation_score',
 'admission_test_score',
 'work_exp_months']

In [84]:
pipe6['preprocessing'].transformers_[1][1].get_feature_names_out()

array(['gender_Male', 'category_General', 'category_NC_OBC',
       'category_SC', 'category_ST', 'graduation_type_Non-Engineer'],
      dtype=object)

In [85]:
col_cat = list(pipe6['preprocessing'].transformers_[1][1].get_feature_names_out())
col_cat

['gender_Male',
 'category_General',
 'category_NC_OBC',
 'category_SC',
 'category_ST',
 'graduation_type_Non-Engineer']

In [86]:
import itertools
col_preproc = list(itertools.chain(col_num, col_cat))

In [87]:
len(col_preproc)

11

In [88]:
pd.DataFrame(data=pipe6['preprocessing'].fit_transform(X_train), columns=col_preproc).head()

,grade_10,grade_12,graduation_score,admission_test_score,work_exp_months,gender_Male,category_General,category_NC_OBC,category_SC,category_ST,graduation_type_Non-Engineer
0,-0.098684,-0.568765,-0.720648,-0.618425,0.944444,1.0,0.0,0.0,1.0,0.0,0.0
1,-0.914474,0.428904,-0.874494,-3.766656,1.444444,0.0,1.0,0.0,0.0,0.0,1.0
2,-1.717105,-0.466200,-1.255061,-1.579853,0.000000,0.0,0.0,1.0,0.0,0.0,1.0
3,0.361842,0.447552,1.157895,-0.867070,0.333333,1.0,1.0,0.0,0.0,0.0,0.0
4,0.677632,0.634033,0.518219,0.517692,1.222222,1.0,1.0,0.0,0.0,0.0,0.0


In [89]:
scores6 = cross_val_score(pipe6, X_train, y6_train, cv=5, scoring="accuracy")
score6 = scores6.mean()

print(scores6)
print(scores6.mean())

[0.72900158 0.74326466 0.76031746 0.73174603 0.75555556]
0.74397705833522


In [90]:
import pickle

# Export pipeline as pickle file
with open("pipeline6.pkl", "wb") as file:
    pickle.dump(pipe6, file)

In [91]:
# Load pipeline from pickle file
my_pipeline6 = pickle.load(open("pipeline6.pkl", "rb"))

my_pipeline6

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_encoder', RobustScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11ddd9b40>),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x11dd8a320>)])),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced'))])

In [92]:
new_data = pd.read_csv("/Users/prasadsawant/Downloads/new_data.csv")

In [93]:
new_data

,gender,category,grade_10,grade_12,graduation_score,graduation_type,admission_test_score,work_exp_months
0,Male,General,68.16,57.58,59.10,Engineer,77.04,26
1,Male,ST,89.75,79.13,92.36,Non-Engineer,94.16,23
2,Female,SC,74.81,81.42,87.84,Non-Engineer,84.27,17
3,Female,ST,91.75,68.73,58.49,Engineer,71.17,2
4,Female,EWS,62.55,79.76,58.94,Non-Engineer,88.75,26
5,Female,SC,69.22,59.70,88.16,Engineer,93.41,19
6,Male,SC,68.38,80.86,78.22,Non-Engineer,94.47,9
7,Male,General,56.94,76.89,69.00,Non-Engineer,61.67,10
8,Male,General,85.90,87.64,95.31,Engineer,95.79,0
9,Female,General,87.70,75.37,78.85,Non-Engineer,60.86,27


In [94]:
predicted_class_6 = my_pipeline6.predict(new_data)
predicted_class_6

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 0])

Predictions

In [95]:
#Predictions
print(predicted_class_1)
print(predicted_class_2)
print(predicted_class_3)
print(predicted_class_4)
print(predicted_class_5)
print(predicted_class_6)

[0 1 0 0 0 0 1 0 0 0]
[0 1 1 0 0 1 1 0 0 0]
[0 1 0 0 0 1 1 0 0 0]
[0 1 1 0 0 0 1 0 0 0]
[0 1 1 1 1 1 1 0 0 0]
[0 1 1 0 1 1 1 0 0 0]


In [96]:
#Accuracy
print(scores1.mean())
print(scores2.mean())
print(scores3.mean())
print(scores4.mean())
print(scores5.mean())
print(scores6.mean())

0.6916227706085076
0.7519125600583603
0.784902774633361
0.7297019092898649
0.7668065303247553
0.74397705833522
